In [1]:
import torch 
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data
import torch.optim as optim

In [2]:
class SMnet(nn.Module):
    def __init__(self):
        super(SMnet, self).__init__()
        self.linear_start1 = nn.Linear(20,128)
        self.linear_start2 = nn.Linear(128,256)
        #self.linear_start1.cuda()
        #self.linear_start2.cuda()
        self.lstm = nn.LSTM(
            input_size=256, 
            hidden_size=512, 
            num_layers=3,
            batch_first=True)
        
       # self.lstm.cuda()
        self.linear1 = nn.Linear(512, 256)
        self.linear2 = nn.Linear(256, 3)
        #self.linear1.cuda()
        #self.linear2.cuda()
        
    def forward(self, robot_sensors):
        ls_out1 = self.linear_start1(robot_sensors)
        ls_out2 = self.linear_start2(ls_out1)
        lstm_out, (h_n, h_c) = self.lstm(ls_out2)
        l_out1 = self.linear1(lstm_out[:,-1,:])
        l_out2 = self.linear2(l_out1)

        return l_out2
        

In [4]:
#from google.colab import files

#uploaded = files.upload()

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt

data_x= pd.read_csv('joints_tobar.txt',delimiter=" ",header=None)
data_y= pd.read_csv('ground_truth.txt',delimiter=" ",header=None)
feat=data_x.as_matrix()
gtruth=data_y.as_matrix()

print('{0:2d} datos de entrada con tamaño {1:2d}'.format(len(feat),len(feat[0])))
print('{0:2d} datos de salida con tamaño {1:2d}'.format(len(gtruth),len(gtruth[0])))
y=[]
x=[]
time=[]
#print(ground_t)
#Extraer x,y,z
for i in range(0,len(feat)):
    xi=[]
    for j in range(1,len(feat[0])):
        xi.append(feat[i][j])
    x.append(xi)
    time.append(feat[i][0])
    #if i%10000==0:
        #print(xi)
        
for i in range(0,len(gtruth)):
    pos=[]
    pos.append(gtruth[i][1])
    pos.append(gtruth[i][2])
    pos.append(gtruth[i][3])
    y.append(pos)
    #if i%10000==0:
        #print(pos)
        
print('{0:2d} datos de entrada con tamaño {1:2d}'.format(len(x),len(x[0])))
print('{0:2d} datos de salida con tamaño {1:2d}'.format(len(y),len(y[0])))
x_train= x[:30]
y_train= y[:30]
x_val = x[30000:]
y_val = y[30000:]
print('tamaño conjunto de entrenamiento: {0:2d}'.format(len(x_train)))
print('tamaño conjunto de validacion: {0:2d}'.format(len(y_val)))

46874 datos de entrada con tamaño 21
46874 datos de salida con tamaño  8
46874 datos de entrada con tamaño 20
46874 datos de salida con tamaño  3
tamaño conjunto de entrenamiento: 30
tamaño conjunto de validacion: 16874


In [7]:
# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)
input= torch.from_numpy(np.array(x_train))
target = torch.from_numpy(np.array(y_train))
#target = target.unsqueeze(0)
print('target size: {}'.format(target.size()))
test_input = torch.from_numpy(np.array(x_val))
test_target = torch.from_numpy(np.array(y_val))
# build the model
seq = SMnet()
seq.double()
criterion = nn.MSELoss()



target size: torch.Size([30, 3])


In [80]:
# create your optimizer
optimizer = optim.SGD(seq.parameters(), lr=0.01)
print(input.size())
#input = input.unsqueeze(0)
input = input.view(-1, 1,20)
print(input.size())
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = seq(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update
print(loss)

torch.Size([30, 1, 20])
torch.Size([30, 1, 20])
tensor(0.0720, dtype=torch.float64, grad_fn=<MseLossBackward>)
